In [7]:
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input

from keras.models import Model
from keras.models import load_model

from keras.layers import Input
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import add
from keras.layers import Embedding

from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.sequence import pad_sequences

from nltk.translate.bleu_score import corpus_bleu

import string
from os import listdir
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pickle import dump
from pickle import load

In [8]:
m = Xception()
m.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [9]:
model = Model(inputs=m.layers[0].input, outputs=m.layers[-2].output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [10]:
import os
print(len(os.listdir('./Flicker8k_Dataset/')))
_ = [print(x) for x in os.listdir('./Flicker8k_Dataset/')[:10]]

8091
1000268201_693b08cb0e.jpg
1001773457_577c3a7d70.jpg
1002674143_1b742ab4b8.jpg
1003163366_44323f5815.jpg
1007129816_e794419615.jpg
1007320043_627395c3d8.jpg
1009434119_febe49276a.jpg
1012212859_01547e3f17.jpg
1015118661_980735411b.jpg
1015584366_dfcec3c85a.jpg


In [ ]:
extractedData = {}
dirName = './Flicker8k_Dataset/'
WIDTH = 299
HEIGHT = 299
CHANNELS = 3
count = 0
for fileName in os.listdir(dirName):
    path = dirName + fileName
    image = load_img(path, target_size=(WIDTH,HEIGHT))
    image = img_to_array(image)
    image.shape = (1,) + image.shape
    image = preprocess_input(image)
    prediction = model.predict(image)
    extractedData[fileName.split('.')[0]] = prediction
    
    print(count, end=' ')
    count += 1

In [ ]:
print(len(extractedData))
# _ = [print(x,y) for x,y in extractedData.items()]

In [ ]:
featureFileName = open('featureXception2.pkl', 'wb')
dump(extractedData, featureFileName)

In [31]:
text = open('./Flickr8k.token.txt', 'r').read()
totalImgDescList = {}

lines = text.split('\n')
print(len(lines))

for line in lines[:-1]:
        
    imageID = line.split()[0]
    imageCount = int(imageID.split('.')[1].split('#')[1])
    imageID = imageID.split('.')[0]
    
    imageDesc =  line.split()[1:]
    imageDesc = [w.lower() for w in imageDesc]
    imageDesc = [w for w in imageDesc if len(w)>1]
    imageDesc = [w for w in imageDesc if w not in list(string.punctuation)]
    imageDesc = [w for w in imageDesc if w.isdigit()==False]
    imageDesc = [w.translate(str.maketrans('','',string.punctuation)) for w in imageDesc]
    imageDesc = ' '.join(imageDesc)
    
    if imageCount == 0:
        totalImgDescList[imageID] = []        
    
    totalImgDescList[imageID].append(imageDesc)
    
print(len(totalImgDescList.keys()))

40460
8092


In [32]:
totalVocabulary = set()
for key in list(totalImgDescList.keys())[:]:
    for d in totalImgDescList[key]:
        totalVocabulary.update(set(d.split()))

print(len(totalVocabulary))

8767


In [33]:
LEN = 3
for i,x in enumerate(list(totalVocabulary)):
    if len(list(x)) == LEN:
        print(i, x)

56 ace
65 aid
112 sun
125 bog
143 bmx
178 gra
186 ski
226 the
263 vie
264 bed
281 are
307 gap
361 wed
397 lav
398 boy
409 you
414 pit
420 dig
439 abs
468 lay
476 how
499 set
500 doe
502 gay
523 mom
606 lap
631 bmw
647 twp
654 ask
708 sky
727 use
731 thw
749 rim
773 min
794 gin
838 zip
864 bum
865 tge
879 dye
880 jog
977 rug
998 nfl
1080 met
1150 dhe
1151 hut
1208 nap
1214 axe
1223 may
1230 zig
1236 fro
1266 4x4
1290 tbe
1360 via
1438 led
1444 leg
1497 gun
1513 two
1539 hte
1567 tip
1669 van
1753 box
1767 art
1779 say
1780 way
1815 gal
1822 day
1852 suv
1901 bag
1923 sat
1936 man
1944 far
1953 bra
1980 web
2035 sad
2080 rag
2081 she
2082 ear
2130 pet
2183 cam
2193 tie
2292 its
2315 bad
2328 dge
2330 god
2332 has
2353 non
2385 bat
2392 mid
2430 pad
2468 bus
2484 fox
2523 who
2588 top
2593 fed
2595 dip
2659 sip
2678 eye
2688 log
2718 net
2799 cry
2842 lip
2912 row
2913 ink
2959 mug
2999 dim
3000 tub
3020 hay
3124 odd
3129 mic
3145 bar
3164 jug
3184 cot
3220 ban
3259 few
3268 dew
3288 dug


In [ ]:
item = []
for ID, des in list(totalImgDescList.items())[:]:
    for d in des:
        item.append(ID + ' ' + d)
        
item = '\n'.join(item)
descFileName = 'imageDescriptionXception2.txt'
with open(descFileName, 'w') as file:
    file.write(item)
    file.close()

In [ ]:
def getIds(fileName):
    with open(fileName, 'r') as textFile:
        text = textFile.read()
        IDs = []
        for imgName in text.split('\n'):
            IDs.append(imgName.split('.')[0])
    return IDs

In [ ]:
def getDescData(fileName, IDs):
    with open(fileName, 'r') as cleanDescFile:
        text = cleanDescFile.read()
        descData = dict()
        for line in text.split('\n'):
            imageID = line.split(' ')[0]
            imageDesc = line.split(' ')[1:]
            imageDesc = ' '.join(imageDesc)
            
            if imageID in IDs:
                if imageID not in descData:
                    descData[imageID] = []
                descData[imageID].append('startSentence ' + imageDesc + ' endSentence')
                    
    return descData

In [ ]:
def getFeatures(IDs):
    completeFeatures = load(open('./featureXception2.pkl', 'rb'))
    features = {i:completeFeatures[i] for i in IDs}
    return features

In [ ]:
trainIDs = getIds('./Flickr_8k.trainImages.txt')
trainDescData = getDescData('./imageDescriptionXception2.txt',trainIDs)
trainFeatures = getFeatures(trainIDs)

print(f'Total Train IDs : {len(set(trainIDs))}')
print(f'Total Train ID:Description : {len(trainDescData)}')
print(f'Total Train ID:Features : {len(trainFeatures)}')

In [ ]:
def getTokenizer(IDs):
    IDs = trainIDs
    cleanDescriptionList = []
    for k in IDs:
        for d in trainDescData[k]:
             cleanDescriptionList.append(d)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(cleanDescriptionList)

    return tokenizer, cleanDescriptionList

In [ ]:
trainTokenizer, trainCleanDescList = getTokenizer(trainIDs)
maxLen = max([len(d.split(' ')) for d in trainCleanDescList[:]])
trainVocabSize = len(trainTokenizer.word_index) + 1

print(maxLen)
print(trainVocabSize)

In [ ]:
def getInputOutput(tokenizer, feature, descList, maxLen, vocabSize):
    input1 = []
    input2 = []
    output = []

    for d in descList:
        sentence = tokenizer.texts_to_sequences([d])[0]
        for i in range(1, len(sentence)):
            input1.append(feature)
            input2.append(pad_sequences([sentence[:i]], maxlen=maxLen)[0])
            output.append(to_categorical([sentence[i]], num_classes=vocabSize)[0])
            
    return np.array(input1), np.array(input2), np.array(output)

In [ ]:
for key in trainIDs[:2]:
    _ = [print(x) for x in trainDescData[key]]
    _ = [print(x) for x in trainFeatures[key]]

In [ ]:
print(maxLen)
print(trainVocabSize)

In [ ]:
dump(trainTokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
def dataGenerator(descData, featureData):
    while 1:
        for key, descList in descData.items():
            feature = featureData[key][0]
            input1, input2, output = getInputOutput(trainTokenizer, feature, descList, maxLen, trainVocabSize)
            yield[[input1, input2], output]

In [ ]:
# generator = dataGenerator(trainDescData, trainFeatures)
# inputs, outputs = next(generator)
# print(inputs[0].shape)
# print(inputs[1].shape)
# print(outputs.shape)

In [ ]:
def model(maxLen, vocabSize):
    input1 = Input(shape=(2048,))
    input1Layer1 = Dropout(0.5)(input1)
    input1Layer2 = Dense(256, activation='relu')(input1Layer1)
    
    input2 = Input(shape=(maxLen,))
    input2Layer1 = Embedding(vocabSize, 256, mask_zero=True)(input2)
    input2Layer2 = Dropout(0.5)(input2Layer1)
    input2Layer3 = LSTM(256)(input2Layer2)
    
    mergeLayer = add([input1Layer2, input2Layer3])
    finalLayer1 = Dense(256, activation='relu')(mergeLayer)
    outputLayer = Dense(vocabSize, activation='softmax')(finalLayer1)
    
    model = Model(inputs=[input1, input2], outputs=outputLayer)
    return model
    

In [ ]:
myModel = model(maxLen, trainVocabSize)
myModel.summary()

In [ ]:
myModel.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
epochs = 20
steps = len(trainDescData)
for i in range(epochs):
    generator = dataGenerator(trainDescData, trainFeatures)
    myModel.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    myModel.save('Xception2_model_' + str(i) + '.h5')

In [ ]:
def generateDesc(model, tokenizer, imageInput):
    mainModel = model
    
    w2i = tokenizer.word_index
    i2w = {j:i for i,j in w2i.items()}

    maxLen = 34
    sentence = nextWord = 'startsentence'

    for _ in range(maxLen):
        textInput = tokenizer.texts_to_sequences([sentence])[0]
        textInput = pad_sequences([textInput], maxlen=maxLen)

        prediction = mainModel.predict([imageInput, textInput])

        nextWord = i2w[np.argmax(prediction)]
        sentence += ' ' + nextWord
        
        if nextWord is None:
            break
        if nextWord == 'endsentence':
            break
            
    return sentence

In [ ]:
def evaluate(model, descriptions, features, tokenizer):
    actual, predicted = list(), list()
    count = 0
    for key, desc_list in descriptions.items():
        count += 1
        if count % 100 == 0:
            print(count, end=' ')
        
        yhat = generateDesc(model, tokenizer, features[key])
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
        
    print('\n\nBLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
testIDs = getIds('./Flickr_8k.testImages.txt')
testDescData = getDescData('./imageDescription.txt',testIDs)
testFeatures = getFeatures(testIDs)

print(f'Total Test IDs : {len(set(testIDs))}')
print(f'Total Test ID:Description : {len(testDescData)}')
print(f'Total Test ID:Features : {len(testFeatures)}')



In [ ]:
for i in range(20):
    model = load_model(f'./Xception2_model_{i}.h5')
    print(f'\n\nMODEL-{i}\n\n')
    evaluate(model, testDescData, testFeatures, trainTokenizer)
    print('\n----------------------------------------\n')